To index data in Azure SQL db, for now we would need to use Azure AI search. 



#### First, let's populate the table with sample data:

1. Update your .env file with required environment variables for connecting to your Azure SQL database (assuming you already have created an Azure SQL resource and have created a database)
2. Ensure you are in the src folder and run: **python init_sql_db.py**
    - Above step will populate a table with sample products and their descriptions

#### Create an Azure AI Search resource (free version is sufficient)
Go to your Azure AI Search resource page:
1. Go to 'Import and Vectorize Data' and click on Azure SQL Database
    - You can choose to also only import your data in case you would like to generate vectors outside of AI Search service. 
2. Populate required info to connect to your SQL database and also choose the 'products_catalogue' table
3. Next step is 'Vectorize your text':
    - Choose description_dup for vectorizing
    - Choose your deployed Azure Open AI embedding model
    - Confirm and click next
4. For this example, we build a once time index so choose 'Once' under scheduled indexing
5. Choose any prefix you want and click on create (we chose 'cat-search')
6. It may take a few minutes for the index to be ready...
    - To check, go to overview -> properties and you can see the state of your index there




#### Remember to update .env values:
- AZURE_SEARCH_ENDPOINT: endpoint is found under url value
- AZURE_SEARCH_KEY: you can find the key value under settings -> key
- AZURE_SEARCH_INDEX: name of your index (which is the prefix you chose. Ours is cat-search)

#### Now we are ready to perform a simple test:

##### Connecting to AI search client:

In [64]:
import os
from dotenv import load_dotenv
load_dotenv()
from azure.search.documents import SearchClient  
from azure.search.documents.models import VectorizableTextQuery 
from azure.core.credentials import AzureKeyCredential 

class AzureSearch():
    def __init__(self):
        self.aisearch_key = os.environ.get("AZURE_SEARCH_KEY")
        self.service_endpoint = os.environ.get("AZURE_SEARCH_ENDPOINT")
        self.index_name = os.environ.get("AZURE_SEARCH_INDEX")
        self.embedding_length = 1536
        self.search_client = SearchClient(self.service_endpoint,
                                          self.index_name,
                                          credential=AzureKeyCredential(self.aisearch_key))

    async def search_query(self, vectors, question, top_n=5):
    
        results = self.search_client.search(search_text=question,
                                            vector_queries=[vectors],
                                            select=[
                                            "prod_id", 
                                            "description",
                                            "name",
                                            "category"],
                                             top=top_n)
        return results
    
    async def hybrid_search(self, search_input):

        vector_query = VectorizableTextQuery(text=search_input, k_nearest_neighbors=2, fields="text_vector", exhaustive=True)

        results_itr = await self.search_query(vector_query, search_input)
        res_list = []
        for r in results_itr:
            res_list.append(r)
        for i, r in enumerate(res_list, 1):
            print(f"{i}. Product ID: {r.get('prod_id', 'N/A')}")
            print(f"   Name: {r.get('name', 'N/A')}")
            print(f"   Category: {r.get('category', 'N/A')}")
            print(f"   Description: {r.get('description', 'N/A')[:100]}...")
            print("-" * 50)

        return res_list

In [65]:
aisearch_engine = AzureSearch()

In [ ]:

question = "a smart watch that can track my health, has a long battery life and is waterproof."


In [63]:
res = await aisearch_engine.hybrid_search(question)

1. Product ID: 1806
   Name: Samsung Galaxy Watch 5
   Category: Wearable
   Description: The Samsung Galaxy Watch 5 redefines smartwatch versatility with a robust suite of health and fitnes...
   Score: 0.0318
--------------------------------------------------
2. Product ID: 20
   Name: Samsung Galaxy Watch 5 Pro
   Category: GPS Watch
   Description: Samsung Galaxy Watch 5 Pro offers robust fitness and health monitoring in a durable titanium build. ...
   Score: 0.0311
--------------------------------------------------
3. Product ID: 1034
   Name: Withings Body Comp Scale
   Category: Health
   Description: Withings Body Comp is a Wi-Fi smart scale that measures body weight, fat percentage, muscle mass, an...
   Score: 0.0167
--------------------------------------------------
4. Product ID: 743
   Name: Apple Watch Ultra
   Category: GPS Watch
   Description: The Apple Watch Ultra is designed for extreme athletes and adventurers, boasting a rugged titanium c...
   Score: 0.0164
-----